# MSCA 32018 Natural Language Processing and Cognitive Computing
## Final Project - Topic Detection 
### Zero-shot (NLI) modeling based on Sentiment Analysis

Shijia Huang

-----

In [1]:
#!pip install -r requirements.txt

In [2]:
# Import basic libraries
import time
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt 

%matplotlib inline

In [3]:
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 500)

In [4]:
# Import NLP libraries
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

from pprint import pprint
import string
from rake_nltk import Rake

import spacy
from spacy import displacy
from spacy.util import minibatch, compounding
spacy.prefer_gpu()
print(spacy.__version__)

import gensim
from gensim import corpora, models
from gensim.utils import simple_preprocess
from gensim.models.ldamulticore import LdaMulticore
from gensim.models import CoherenceModel

import pyLDAvis
import pyLDAvis.gensim as gensimvis
#import pyLDAvis.gensim_models as gensimvis
pyLDAvis.enable_notebook()

import tensorflow as tf
from transformers import pipeline

2023-05-15 12:29:45.075215: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


3.5.2


In [5]:
import multiprocessing as mp

num_processors = mp.cpu_count()
print(f'Available CPUs: {num_processors}')

Available CPUs: 12


### Read New Articles with Sentiment Score

In [6]:
%%time

# GCP version
path = "gs://nlp-final-project-data/data/"
df_news = pd.read_parquet(path + 'news_sentiment.parquet', engine='pyarrow')
df_news.shape

CPU times: user 1min 29s, sys: 41.4 s, total: 2min 10s
Wall time: 3min 32s


(154283, 11)

In [7]:
df_news.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 154283 entries, 0 to 154282
Data columns (total 11 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   id                154283 non-null  int64 
 1   date              154283 non-null  object
 2   cleaned title     154283 non-null  object
 3   cleaned text      154283 non-null  object
 4   title_tokens      154283 non-null  object
 5   title_lemmatized  154283 non-null  object
 6   text_tokens       154283 non-null  object
 7   text_lemmatized   154283 non-null  object
 8   title_keywords    154283 non-null  object
 9   text_keywords     154283 non-null  object
 10  sentiment         154283 non-null  int64 
dtypes: int64(2), object(9)
memory usage: 12.9+ MB


In [8]:
df_news.head(2)

,id,date,cleaned title,cleaned text,title_tokens,title_lemmatized,text_tokens,text_lemmatized,title_keywords,text_keywords,sentiment
0,1,2020-02-27,Children With Autism Saw Their Learning and Social Skills Boosted After Playing With This AI Robot,"Children With Autism Saw Their Learning and Social Skills Boosted After Playing With This AI Robot admin Latest posts by admin see all Mansplaining in conferences: How can we get him to forestall February 27, 2020 Coronavirus Could Explode in the U.S. Overnight Like it Did in Italy February 27, 2020 Levi Strauss marks the next phase in corporate paid leave policies February 27, 2020 Scientists who designed an artificially clever robotic that helped youngsters with autism spice up their ...","[children, autism, saw, learning, social, skills, boosted, playing, ai, robot, children, autism, saw, learning, social, skills, boosted, playing, ai, robot, children, autism, saw, learning, social, skills, boosted, playing, ai_robot]","[child, autism, see, learn, social, skill, boost, play, robot, child, autism, see, learn, social, skill, boost, play, robot, child, autism, see, learn, social, skill, boost, play, ai_robot]","[children, autism, saw, learning, social, skills, boosted, playing, ai, robot, admin, latest, posts, admin, see, mansplaining, conferences, get, forestall, february, coronavirus, could, explode, overnight, like, italy, february, levi, strauss, marks, next, phase, corporate, paid, leave, policies, february, scientists, designed, artificially, clever, robotic, helped, youngsters, autism, spice, studying, social, talents, hope, era, may, future, help, others, developmental, dysfunction, learn, ...","[child, autism, see, learn, social, skill, boost, play, robot, late, post, admin, see, mansplaining, conference, get, explode, overnight, mark, next, phase, corporate, pay, leave, policy, scientist, design, artificially, clever, robotic, help, youngster, autism, spice, study, social, talent, era, future, help, other, developmental, dysfunction, learn, notice, youngster, gentle, average, autism, take, domestic, s, refer, socially, assistive, robotic, name, kiwi, month, accord, commentary, way...","[social, skill, see, play, learn, child, boost, autism, robot, ai_robot]","[robotic, youngster, kid, child, kiwi, market, autism, learn, crew, talent]",5
1,2,2021-03-26,"Forget ML, AI and Industry 4.0 – obsolescence should be your focus","Forget ML, AI and Industry 4.0 obsolescence should be your focus The world entered a new era of accelerated transformation in the last eighteen months that will continue to evolve and press forward for years to come. Most businesses are playing catchup trying to make sense of a new timeline where the ten years that had been set aside for careful planning and implementation of what was coming up next no longer exists. The next is happening now and, regardless of your industry or seniority, t...","[forget, ml, ai, industry, obsolescence, focus, forget, ml, ai, industry, obsolescence, focus, forget, ml, ai, industry, obsolescence, focus]","[forget, ml, ai, industry, obsolescence, focus, forget, ml, ai, industry, obsolescence, focus, forget, ml, ai, industry, obsolescence, focus]","[forget, ml, ai, industry, obsolescence, focus, world, entered, new, era, accelerated, transformation, last, eighteen, months, continue, evolve, press, forward, years, come, businesses, playing, catchup, trying, make, sense, new, timeline, ten, years, set, aside, careful, planning, implementation, coming, next, longer, exists, next, happening, regardless, industry, seniority, status, quo, shifted, better, face, back, invited, attend, pompous, meeting, london, brazilian, embassy, along, selec...","[forget, ai, industry, obsolescence, focus, world, enter, new, era, accelerate, transformation, last, month, continue, evolve, press, forward, year, come, business, play, catchup, try, make, sense, new, timeline, year, set, aside, careful, planning, implementation, come, next, long, exist, ne

In [9]:
### SAMPLE DATA
# df_news = df_news.sample(frac=0.01, random_state=42)
df_news.shape

(154283, 11)

# Select articles with positive and negative sentiment scores

- Positive: sentiment score > 3
- Negative: sentiment score < 3

In [10]:
# select positive sentiment articles
df_news_positive = df_news[df_news['sentiment'] > 3]
df_news_positive.shape

(104228, 11)

In [13]:
# print a sample
df_news_positive.sample(1)

,id,date,cleaned title,cleaned text,title_tokens,title_lemmatized,text_tokens,text_lemmatized,title_keywords,text_keywords,sentiment
3583,4650,2023-03-16,Microsoft unveils AI office copilot in fast-moving race with Google,"Microsoft unveils AI office copilot in fastmoving race with Google 2 min read . Updated: 16 Mar 2023, 10:52 PM IST The technology company previewed a new AI copilot for Microsoft 365 The new product suite includes Word documents, Excel spreadsheets, PowerPoint presentations and Outlook emails Microsoft Corp on Thursday trumpeted its latest plans to put artificial intelligence into the hands of more users, answering a spate of unveilings this week by its rival Google with upgrades to its...","[microsoft, unveils, ai, office, copilot, fastmoving, race, google, microsoft_unveils, ai, office_copilot, fastmoving_race, google, microsoft_unveils, ai, office_copilot, fastmoving_race, google]","[microsoft, unveil, ai, office, copilot, fastmove, race, google, microsoft_unveil, ai, fastmoving_race, google, microsoft_unveil, ai, fastmoving_race, google]","[microsoft, unveils, ai, office, copilot, fastmoving, race, google, min, read, updated, mar, pm, ist, technology, company, previewed, new, ai, copilot, microsoft, new, product, suite, includes, word, documents, excel, spreadsheets, powerpoint, presentations, outlook, emails, microsoft, corp, thursday, trumpeted, latest, plans, put, artificial, intelligence, hands, users, answering, spate, unveilings, week, rival, google, upgrades, widely, used, office, software, technology, company, previewe...","[unveil, ai, office, copilot, fastmove, race, read, update, technology, company, preview, new, copilot, new, product, suite, include, word, document, excel, spreadsheet, powerpoint, presentation, outlook, email, trumpet, late, plan, put, artificial, intelligence, hand, user, answer, spate, unveiling, week, rival, google, upgrade, widely, use, office, software, technology, company, preview, new, copilot, microsoft, product, suite, include, word, document, excel, spreadsheet, powerpoint, prese...","[google, ai, microsoft_unveil, fastmoving_race, unveil, race, office, microsoft, fastmove, copilot]","[ai, new, say, business, update, software, company, include, week, application]",4


In [11]:
# select negative sentiment articles
df_news_negative = df_news[df_news['sentiment'] < 3]
df_news_negative.shape

(14932, 11)

In [14]:
# print a sample
df_news_negative.sample(1)

,id,date,cleaned title,cleaned text,title_tokens,title_lemmatized,text_tokens,text_lemmatized,title_keywords,text_keywords,sentiment
81573,106101,2023-03-26,Up to 80 percent of workers could see jobs impacted by AI,"Up to 80 percent of workers could see jobs impacted by AI Researchers from OpenAI and the University of Pennsylvania argued in a new research paper that AI could soon be shaking up some fields following the rise of ChatGPT, a shockingly intelligent chatbotreleased in November. Researchers argued that 80 percent of the US workforce could have at least 10 percent of their work tasks affected by the introduction of ChatGPT. They also found that about 19 percent of workers may find at least 5...","[percent, workers, could, see, jobs, impacted, ai, percent, workers, could_see, jobs_impacted, ai, percent, workers, could_see, jobs_impacted, ai]","[percent, worker, see, job, impact, ai, percent, worker, could_see, jobs_impacte, ai, percent, worker, could_see, jobs_impacte, ai]","[percent, workers, could, see, jobs, impacted, ai, researchers, openai, university, pennsylvania, argued, new, research, paper, ai, could, soon, shaking, fields, following, rise, chatgpt, shockingly, intelligent, chatbotreleased, november, researchers, argued, percent, us, workforce, could, least, percent, work, tasks, affected, introduction, chatgpt, also, found, percent, workers, may, find, least, percent, duties, impacted, gpt, generalpurpose, technologies, researchers, also, found, highe...","[percent, worker, see, job, impact, ai, researcher, argue, new, research, paper, ai, soon, shake, field, follow, rise, chatgpt, shockingly, intelligent, chatbotrelease, researcher, argue, percent, least, percent, work, task, affect, introduction, chatgpt, also, find, percent, worker, find, least, percent, duty, impact, gpt, generalpurpose, technology, researcher, also, find, higherincome, job, likely, great, exposure, gpt, span, almost, industry, researcher, argue, percent, least, percent, w...","[worker, percent, ai, jobs_impacte, could_see, see, job, impact]","[task, exposure, researcher, percent, work, gpt, worker, ai, find, time]",1


## Topic Modeling - Zero-shot (NLI) modeling

Using candidate labels from LDA model with n=18 topics

In [ ]:
labels_positive = []

In [ ]:
labels_negative = []

#### Check for GPU presence

In [11]:
#Verify we got CPU + GPU or only CPU
tf.config.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]

In [12]:
!nvidia-smi

/bin/bash: nvidia-smi: command not found


In [ ]:
tf.__version__

'2.6.4'

### Modelling

In [13]:
# Modeling
classifier = pipeline(task="zero-shot-classification", 
                      model="facebook/bart-large-mnli",
                      device=0)

AssertionError: Torch not compiled with CUDA enabled